In [1]:
import sys
sys.path.append("../../smsl")

In [2]:
from smsl.graphBigtraj import BigTrajGraphAgent
from smsl.config import ConfAgent
from smsl.graph import PR1

In [3]:
import os
import pandas as pd
import numpy as np
from IPython.display import JSON

In [4]:
class HotSpotAgent(dict, ConfAgent):
    def __init__(self, MAgent):
        ConfAgent.__init__(self)
        self.end_ns = self.time_num
        self.framesperns = self.frame_num // self.time_num
        self.MAgent = MAgent
    def load_systems(self, system2offical_name):
        ori_path = os.getcwd()
        for system, system_offical_name in system2offical_name.items():
            os.chdir(f'../../{system}/d.fluctmatch')
            b_agent = BigTrajGraphAgent(self.MAgent)
            b_agent.system_offical_name = system_offical_name
            self[system] = b_agent
        os.chdir(ori_path)
    def get_hotspot(self, verbose=0, is_return=False):
        self.m_abbr = self.get_m_abbr() 
        dict_hotspot = {self.m_abbr:{}}
        for system, b_agent in self.items():
            dict_hotspot[b_agent.m_abbr][system] = {}
            dict_hotspot[b_agent.m_abbr][system] = get_hotspot_by_graph_agent(b_agent.t_agent, n_bp=self.n_bp, verbose=verbose)
        if is_return:
            return dict_hotspot
        else:
            self.dict_hotspot = dict_hotspot
    def get_m_abbr(self):
        return list(self.values())[0].m_abbr ## first system of sub group name
    def get_df_k_with_hotspots(self, k_value='k_avg_std', is_return=False): ## for s = pr1
        list_k_with_hotspots = []
        for s, dict_subgroup in self.dict_hotspot.items(): 
            for system, dict_system in dict_subgroup.items():
                for position, dict_position in dict_system.items():
                    pos_seq = position
                    k_avg = dict_position['hotspot']['avg']['avg']
                    k_std = dict_position['hotspot']['avg']['std']
                    list_k_with_hotspots.append({
                        'system'   : self[system].system_offical_name,
                        'sequence' : pos_seq,
                        'k_avg'    : k_avg,
                        'k_std'    : k_std,
                        'k_avg_std': f'{k_avg:.2f} ± {k_std:.2f}',
                    })
        df_k_with_hotspots_1 = pd.DataFrame(list_k_with_hotspots)
        df_k_with_hotspots = df_k_with_hotspots_1.set_index(['sequence'])
        df_k_with_hotspots = df_k_with_hotspots.pivot(columns='system', values=k_value)

        ## sort the row and column index
        sort_system  = list(dict.fromkeys(df_k_with_hotspots_1['system']))
        sort_seq     = list(dict.fromkeys(df_k_with_hotspots_1['sequence']))
        df_k_with_hotspots = df_k_with_hotspots.loc[sort_seq, sort_system]
        if is_return:
            return df_k_with_hotspots
        else:
            self.df_k_with_hotspots = df_k_with_hotspots 

In [5]:
def generate_cluster_hotspots(self): ## for s = pr1
    cluster2hotspots = {
        'all' : {i:['rank',i] for i in range(1, 3+1)}, ## rank1..rank3
    }
    return cluster2hotspots

## SmallTimeAgent level
def get_cluster(self, resid_i, resid_j): ## for s = pr1
    return 'all'

## SmallTimeAgent level
def generate_df_position2hotspots(self, resid_i, resid_j, hotspots): ## for s = pr1
    dict_ks = {}
    df_m = self.mean.df_m
    mask1 = (df_m['Resid_i']==resid_i) & (df_m['Resid_j']==resid_j)
    df_resid_ij = df_m[mask1]
    for i, hotspot in hotspots.items():
        if isinstance(hotspot[1], int): ## rank n: ['rank', 1]
            df_sele = df_resid_ij.sort_values(by='k', ascending=False).iloc[[hotspot[1]-1]]
        elif isinstance(hotspot[1], str): ## hotspot: ['N1', 'O6']
            mask2 = (df_resid_ij['Atomname_i']==hotspot[0]) & (df_resid_ij['Atomname_j']==hotspot[1])
            df_sele = df_resid_ij[mask2]
        else:
            raise RuntimeError('Could not recognize cluster2hotspots:', cluster2hotspots)
        ## generate dict_ks
        pair_id, ser_sele = df_sele.index, df_sele.iloc[0]
        dict_ks[i] = {
            'query'     : ''.join(map(str, hotspot)),
            'atomname_i': ser_sele.Atomname_j,
            'atomname_j': ser_sele.Atomname_i,
            'k'         : {},
        }
        for time_label, st_agent in self.items():
            dict_ks[i]['k'][time_label] = st_agent.df_m.loc[pair_id].squeeze().k
        ks = pd.Series(dict_ks[i]['k'])
        dict_ks[i]['k']['avg'] = ks.mean()
        dict_ks[i]['k']['std'] = ks.std() ## Note: numpy.std(ddof=0), but pandas.std(ddof=1)
    ## k_avg in {aiaj}^s
    dict_ks['avg']  = {}
    for time_label, _ in self.items():
        ks = []
        for i, hotspot in hotspots.items():
            ks.append(dict_ks[i]['k'][time_label])
        dict_ks['avg'][time_label] = np.mean(ks)
    ks = pd.Series(dict_ks['avg'])
    dict_ks['avg']['avg'] = ks.mean()
    dict_ks['avg']['std'] = ks.std() ## Note: numpy.std(ddof=0), but pandas.std(ddof=1)
    return dict_ks

## GraphAgent level, for s = pr1
def get_hotspot_by_graph_agent(self, n_bp, verbose=0):
    cluster2hotspots = generate_cluster_hotspots(self)
    dict_position = {}
    seq_shift = self.sequence[0].find('GGG') - 1 ## for sequence != wtTel22 use
    for resid_i_align in range(1, n_bp):  ## ex: 'G4' in wtTel26(='G2' in wtTel22), resid_i=4, resid_i_align=2, resname_i_align='G'
        resid_j_align   = resid_i_align + 1
        resid_i = resid_i_align + seq_shift
        resid_j = resid_j_align + seq_shift
        resname_i_align = self.strandid2sequence['STRAND1'][resid_i]
        resname_j_align = self.strandid2sequence['STRAND1'][resid_j]
        sequence = f'{resname_i_align}{resid_i_align}{resname_j_align}{resid_j_align}'
        ## generate dict_hotspot
        cluster = get_cluster(self.mean, resid_i, resid_j)
        hotspots = cluster2hotspots[cluster] 
        if verbose:
            print(self.system, sequence, resid_i, resid_j, cluster, hotspots)
        dict_ks = generate_df_position2hotspots(self, resid_i, resid_j, hotspots)
        dict_position[sequence] = {
            'resid_i': int(resid_i), ## save json format from int istead of np.int64(cannot save) 
            'resid_j': int(resid_j),
            'cluster': cluster,
            'hotspot': dict_ks
        }
    return dict_position

In [6]:
system2offical_name = {
'propeller': 'Propeller',
'basket'   : 'Basket',
'chair'    : 'Chair',
'hybrid-i' : 'Hybrid-I',
'hybrid-ii': 'Hybrid-II',
# 'dsdna'    : 'dsDNA',
}

In [7]:
hs_agent = HotSpotAgent(PR1)
hs_agent.load_systems(system2offical_name)

Reading data from: /home/user/workfolder/g4dna_protocol_demo/propeller/d.fluctmatch/bigtraj_fluctmatch/0000_0500/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/propeller/d.fluctmatch/bigtraj_fluctmatch/0250_0750/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/propeller/d.fluctmatch/bigtraj_fluctmatch/0500_1000/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/propeller/d.fluctmatch/bigtraj_fluctmatch/0750_1250/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/propeller/d.fluctmatch/bigtraj_fluctmatch/1000_1500/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/propeller/d.fluctmatch/bigtraj_fluctmatch/1250_1750/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/propeller/d.fluctmatch/bigtraj_fl

/home/user/anaconda3/envs/fluctmatch/lib/python3.8/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: A
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/home/user/anaconda3/envs/fluctmatch/lib/python3.8/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: A
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/home/user/anaconda3/envs/fluctmatch/lib/python3.8/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: A
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/home/user/anaconda3/envs/fluctmatch/lib/python3.8/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: A
  warnings.warn("Failed to 

Reading data from: /home/user/workfolder/g4dna_protocol_demo/propeller/d.fluctmatch/bigtraj_fluctmatch/1750_2250/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/propeller/d.fluctmatch/bigtraj_fluctmatch/2000_2500/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/propeller/d.fluctmatch/bigtraj_fluctmatch/2250_2750/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/propeller/d.fluctmatch/bigtraj_fluctmatch/2500_3000/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/propeller/d.fluctmatch/bigtraj_fluctmatch/2750_3250/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/propeller/d.fluctmatch/bigtraj_fluctmatch/3000_3500/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/propeller/d.fluctmatch/bigtraj_fl

/home/user/anaconda3/envs/fluctmatch/lib/python3.8/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: A
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/home/user/anaconda3/envs/fluctmatch/lib/python3.8/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: A
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/home/user/anaconda3/envs/fluctmatch/lib/python3.8/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: A
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/home/user/anaconda3/envs/fluctmatch/lib/python3.8/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: A
  warnings.warn("Failed to 

Reading data from: /home/user/workfolder/g4dna_protocol_demo/propeller/d.fluctmatch/bigtraj_fluctmatch/4000_4500/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/propeller/d.fluctmatch/bigtraj_fluctmatch/4250_4750/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/propeller/d.fluctmatch/bigtraj_fluctmatch/4500_5000/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/propeller/d.fluctmatch/bigtraj_fluctmatch/0000_5000/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/basket/d.fluctmatch/bigtraj_fluctmatch/0000_0500/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/basket/d.fluctmatch/bigtraj_fluctmatch/0250_0750/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/basket/d.fluctmatch/bigtraj_fluctmatch/

...

/home/user/anaconda3/envs/fluctmatch/lib/python3.8/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: A
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/home/user/anaconda3/envs/fluctmatch/lib/python3.8/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: A
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/home/user/anaconda3/envs/fluctmatch/lib/python3.8/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: A
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/home/user/anaconda3/envs/fluctmatch/lib/python3.8/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: A
  warnings.warn("Failed to 

Reading data from: /home/user/workfolder/g4dna_protocol_demo/hybrid-i/d.fluctmatch/bigtraj_fluctmatch/0000_5000/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/hybrid-ii/d.fluctmatch/bigtraj_fluctmatch/0000_0500/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/hybrid-ii/d.fluctmatch/bigtraj_fluctmatch/0250_0750/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/hybrid-ii/d.fluctmatch/bigtraj_fluctmatch/0500_1000/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/hybrid-ii/d.fluctmatch/bigtraj_fluctmatch/0750_1250/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/hybrid-ii/d.fluctmatch/bigtraj_fluctmatch/1000_1500/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/hybrid-ii/d.fluctmatch/bigtraj_flu

/home/user/anaconda3/envs/fluctmatch/lib/python3.8/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: A
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/home/user/anaconda3/envs/fluctmatch/lib/python3.8/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: A
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/home/user/anaconda3/envs/fluctmatch/lib/python3.8/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: A
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/home/user/anaconda3/envs/fluctmatch/lib/python3.8/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: A
  warnings.warn("Failed to 

Reading data from: /home/user/workfolder/g4dna_protocol_demo/hybrid-ii/d.fluctmatch/bigtraj_fluctmatch/2000_2500/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/hybrid-ii/d.fluctmatch/bigtraj_fluctmatch/2250_2750/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/hybrid-ii/d.fluctmatch/bigtraj_fluctmatch/2500_3000/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/hybrid-ii/d.fluctmatch/bigtraj_fluctmatch/2750_3250/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/hybrid-ii/d.fluctmatch/bigtraj_fluctmatch/3000_3500/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/hybrid-ii/d.fluctmatch/bigtraj_fluctmatch/3250_3750/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/hybrid-ii/d.fluctmatch/bigtraj_fl

/home/user/anaconda3/envs/fluctmatch/lib/python3.8/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: A
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/home/user/anaconda3/envs/fluctmatch/lib/python3.8/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: A
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/home/user/anaconda3/envs/fluctmatch/lib/python3.8/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: A
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/home/user/anaconda3/envs/fluctmatch/lib/python3.8/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: A
  warnings.warn("Failed to 

In [8]:
hs_agent.get_hotspot()
hs_agent.get_df_k_with_hotspots()

In [9]:
JSON(hs_agent.dict_hotspot)

<IPython.core.display.JSON object>

In [10]:
hs_agent.df_k_with_hotspots

system,Propeller,Basket,Chair,Hybrid-I,Hybrid-II
sequence,,,,,
A1G2,6.81 ± 3.90,14.45 ± 2.94,9.35 ± 2.23,10.83 ± 4.39,3.29 ± 0.52
G2G3,13.34 ± 1.61,12.46 ± 0.93,17.43 ± 0.26,16.11 ± 0.66,19.37 ± 0.38
G3G4,5.85 ± 1.39,15.53 ± 4.21,9.89 ± 3.53,8.03 ± 0.61,7.43 ± 1.12
G4T5,7.72 ± 2.04,13.36 ± 2.40,9.62 ± 3.95,7.97 ± 1.36,9.03 ± 3.82
T5T6,5.46 ± 6.81,11.25 ± 3.14,11.78 ± 4.34,15.05 ± 4.63,11.50 ± 4.66
T6A7,13.35 ± 3.83,10.99 ± 1.44,7.64 ± 2.43,12.73 ± 2.91,4.92 ± 0.64
A7G8,7.94 ± 4.42,4.71 ± 1.64,10.91 ± 0.73,9.70 ± 1.89,9.06 ± 1.30
G8G9,12.05 ± 2.15,13.49 ± 0.97,14.24 ± 2.39,16.46 ± 0.50,18.72 ± 0.61
G9G10,4.42 ± 1.96,2.71 ± 1.63,15.21 ± 0.87,6.51 ± 1.03,15.45 ± 2.66


In [11]:
system2offical_name = {
'propeller': 'Propeller',
'basket'   : 'Basket',
'chair'    : 'Chair',
'hybrid-i' : 'Hybrid-I',
'hybrid-ii': 'Hybrid-II',
# 'dsdna'    : 'dsDNA',
'basket_na': 'Basket-Na',
'hybrid-ii_wttel26'    : 'Hybrid-II-wtTel26',
'propeller_poh'        : 'Propeller-POH',
'propeller_ewv'        : 'Propeller-EWV', 
'hybrid-ii_wttel26_ewv': 'Hybrid-II-wtTel26-EWV', 
'hybrid-ii_wttel26_poh': 'Hybrid-II-wtTel26-POH', 
}

In [12]:
hs_agent = HotSpotAgent(PR1)
hs_agent.load_systems(system2offical_name)
hs_agent.get_hotspot()
hs_agent.get_df_k_with_hotspots()

Reading data from: /home/user/workfolder/g4dna_protocol_demo/propeller/d.fluctmatch/bigtraj_fluctmatch/0000_0500/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/propeller/d.fluctmatch/bigtraj_fluctmatch/0250_0750/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/propeller/d.fluctmatch/bigtraj_fluctmatch/0500_1000/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/propeller/d.fluctmatch/bigtraj_fluctmatch/0750_1250/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/propeller/d.fluctmatch/bigtraj_fluctmatch/1000_1500/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/propeller/d.fluctmatch/bigtraj_fluctmatch/1250_1750/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/propeller/d.fluctmatch/bigtraj_fl

/home/user/anaconda3/envs/fluctmatch/lib/python3.8/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: A
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/home/user/anaconda3/envs/fluctmatch/lib/python3.8/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: A
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/home/user/anaconda3/envs/fluctmatch/lib/python3.8/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: A
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/home/user/anaconda3/envs/fluctmatch/lib/python3.8/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: A
  warnings.warn("Failed to 

Reading data from: /home/user/workfolder/g4dna_protocol_demo/propeller/d.fluctmatch/bigtraj_fluctmatch/2000_2500/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/propeller/d.fluctmatch/bigtraj_fluctmatch/2250_2750/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/propeller/d.fluctmatch/bigtraj_fluctmatch/2500_3000/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/propeller/d.fluctmatch/bigtraj_fluctmatch/2750_3250/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/propeller/d.fluctmatch/bigtraj_fluctmatch/3000_3500/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/propeller/d.fluctmatch/bigtraj_fluctmatch/3250_3750/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/propeller/d.fluctmatch/bigtraj_fl

/home/user/anaconda3/envs/fluctmatch/lib/python3.8/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: A
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/home/user/anaconda3/envs/fluctmatch/lib/python3.8/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: A
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/home/user/anaconda3/envs/fluctmatch/lib/python3.8/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: A
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/home/user/anaconda3/envs/fluctmatch/lib/python3.8/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: A
  warnings.warn("Failed to 

Reading data from: /home/user/workfolder/g4dna_protocol_demo/propeller/d.fluctmatch/bigtraj_fluctmatch/4250_4750/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/propeller/d.fluctmatch/bigtraj_fluctmatch/4500_5000/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/propeller/d.fluctmatch/bigtraj_fluctmatch/0000_5000/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/basket/d.fluctmatch/bigtraj_fluctmatch/0000_0500/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/basket/d.fluctmatch/bigtraj_fluctmatch/0250_0750/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/basket/d.fluctmatch/bigtraj_fluctmatch/0500_1000/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/basket/d.fluctmatch/bigtraj_fluctmatch/075

...

/home/user/anaconda3/envs/fluctmatch/lib/python3.8/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: A
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/home/user/anaconda3/envs/fluctmatch/lib/python3.8/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: A
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/home/user/anaconda3/envs/fluctmatch/lib/python3.8/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: A
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/home/user/anaconda3/envs/fluctmatch/lib/python3.8/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: A
  warnings.warn("Failed to 

Reading data from: /home/user/workfolder/g4dna_protocol_demo/hybrid-ii_wttel26_poh/d.fluctmatch/bigtraj_fluctmatch/2000_2500/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/hybrid-ii_wttel26_poh/d.fluctmatch/bigtraj_fluctmatch/2250_2750/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/hybrid-ii_wttel26_poh/d.fluctmatch/bigtraj_fluctmatch/2500_3000/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/hybrid-ii_wttel26_poh/d.fluctmatch/bigtraj_fluctmatch/2750_3250/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/hybrid-ii_wttel26_poh/d.fluctmatch/bigtraj_fluctmatch/3000_3500/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/hybrid-ii_wttel26_poh/d.fluctmatch/bigtraj_fluctmatch/3250_3750/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /ho

/home/user/anaconda3/envs/fluctmatch/lib/python3.8/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: A
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/home/user/anaconda3/envs/fluctmatch/lib/python3.8/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: A
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/home/user/anaconda3/envs/fluctmatch/lib/python3.8/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: A
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/home/user/anaconda3/envs/fluctmatch/lib/python3.8/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: A
  warnings.warn("Failed to 

Reading data from: /home/user/workfolder/g4dna_protocol_demo/hybrid-ii_wttel26_poh/d.fluctmatch/bigtraj_fluctmatch/4000_4500/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/hybrid-ii_wttel26_poh/d.fluctmatch/bigtraj_fluctmatch/4250_4750/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/hybrid-ii_wttel26_poh/d.fluctmatch/bigtraj_fluctmatch/4500_5000/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/hybrid-ii_wttel26_poh/d.fluctmatch/bigtraj_fluctmatch/0000_5000/result/pairtypes_k_b0_cutoff_5.00.csv


/home/user/anaconda3/envs/fluctmatch/lib/python3.8/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: A
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/home/user/anaconda3/envs/fluctmatch/lib/python3.8/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: A
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/home/user/anaconda3/envs/fluctmatch/lib/python3.8/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: A
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))


In [13]:
hs_agent.df_k_with_hotspots

system,Propeller,Basket,Chair,Hybrid-I,Hybrid-II,Basket-Na,Hybrid-II-wtTel26,Propeller-POH,Propeller-EWV,Hybrid-II-wtTel26-EWV,Hybrid-II-wtTel26-POH
sequence,,,,,,,,,,,
A1G2,6.81 ± 3.90,14.45 ± 2.94,9.35 ± 2.23,10.83 ± 4.39,3.29 ± 0.52,16.05 ± 1.46,8.34 ± 1.73,3.01 ± 2.05,4.43 ± 1.98,7.28 ± 0.93,5.38 ± 0.69
G2G3,13.34 ± 1.61,12.46 ± 0.93,17.43 ± 0.26,16.11 ± 0.66,19.37 ± 0.38,11.78 ± 2.42,17.62 ± 1.82,12.78 ± 1.74,11.83 ± 1.90,15.80 ± 1.72,16.81 ± 2.69
G3G4,5.85 ± 1.39,15.53 ± 4.21,9.89 ± 3.53,8.03 ± 0.61,7.43 ± 1.12,7.88 ± 6.13,5.77 ± 2.39,6.36 ± 2.08,6.34 ± 0.59,11.28 ± 4.99,8.10 ± 5.91
G4T5,7.72 ± 2.04,13.36 ± 2.40,9.62 ± 3.95,7.97 ± 1.36,9.03 ± 3.82,5.06 ± 4.30,13.03 ± 2.96,5.84 ± 1.85,7.39 ± 1.37,9.37 ± 2.28,6.78 ± 4.43
T5T6,5.46 ± 6.81,11.25 ± 3.14,11.78 ± 4.34,15.05 ± 4.63,11.50 ± 4.66,10.66 ± 5.11,13.12 ± 4.46,7.17 ± 3.98,4.36 ± 1.84,2.27 ± 2.35,6.30 ± 6.00
T6A7,13.35 ± 3.83,10.99 ± 1.44,7.64 ± 2.43,12.73 ± 2.91,4.92 ± 0.64,5.62 ± 3.09,9.79 ± 1.12,15.19 ± 2.50,13.54 ± 1.58,9.00 ± 0.58,9.18 ± 1.14
A7G8,7.94 ± 4.42,4.71 ± 1.64,10.91 ± 0.73,9.70 ± 1.89,9.06 ± 1.30,7.33 ± 1.95,10.58 ± 0.41,4.83 ± 2.35,8.68 ± 4.32,8.12 ± 1.49,8.96 ± 1.56
G8G9,12.05 ± 2.15,13.49 ± 0.97,14.24 ± 2.39,16.46 ± 0.50,18.72 ± 0.61,13.05 ± 2.93,16.23 ± 0.64,13.07 ± 2.47,12.67 ± 2.98,14.43 ± 1.17,15.66 ± 1.51
G9G10,4.42 ± 1.96,2.71 ± 1.63,15.21 ± 0.87,6.51 ± 1.03,15.45 ± 2.66,6.20 ± 3.14,15.39 ± 2.55,5.36 ± 1.71,5.92 ± 2.59,17.25 ± 1.16,15.45 ± 1.83


### Save all k value data in json format

In [14]:
import json, copy
for subgroup, data in hs_agent.dict_hotspot.items():
    data_copy = copy.deepcopy(data)
    for system, d1 in data_copy.items():
        d1_keys = list(d1.keys())
        for position in d1_keys:
            if isinstance(position, tuple):
                link_position = '-'.join(position)
                d1[link_position] = d1.pop(position)
    with open(f'data/{subgroup}.json', 'w') as f:
        json.dump(data_copy, f)

### Save k value with specific position in csv file

In [15]:
df_ks_position = hs_agent.get_df_k_with_hotspots(k_value='k_avg', is_return=True)
df_ks_position.to_csv(f'data/{hs_agent.m_abbr}_avg.csv')

In [16]:
df_ks_position = hs_agent.get_df_k_with_hotspots(k_value='k_std', is_return=True)
df_ks_position.to_csv(f'data/{hs_agent.m_abbr}_std.csv')